In [1]:
import napari
from qtpy.QtWidgets import QSlider, QVBoxLayout, QWidget, QLabel
from qtpy.QtCore import Qt

import sys
import os
import importlib

parent = os.path.dirname(os.getcwd())
sys.path.insert(0, parent)

from Helpers import loadData


In [ ]:
folderData = os.path.join(parent,'Data')

In [4]:
#data = loadData(location=folderData, fileName='convMap_17.hdf5')
data = loadData(location=folderData, fileName='downscale_17.hdf5')

In [5]:
viewer = napari.Viewer()
viewer.add_image(data)
viewer.show()